In [1]:
import datetime
import time
import uuid

import numpy as np
# package(s) for data handling
import pandas as pd
import shapely.geometry
import simpy

import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}

In [2]:
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasMultiContainer,
        core.HasResource,
    ),
    {},
)

TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.MultiContainerDependentMovable,
        core.Processor,
        core.HasResource,
    ),
    {"key": "MultiStoreHopper"},
)

In [3]:
location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)
location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)

from_site = Site(
    env=my_env,
    name="Winlocatie",
    ID=str(uuid.uuid4()),
    geometry=location_from_site,
    store_capacity=4,
    nr_resources=1,
    initials=[
        {"id": "MP", "level": 10, "capacity": 10},
    ],
)
to_site = Site(
    env=my_env,
    name="Dumplocatie",
    ID=str(uuid.uuid4()),
    geometry=location_to_site,
    store_capacity=4,
    nr_resources=1,
    initials=[
        {"id": "MP", "level": 0, "capacity": 10},
    ],
)

In [4]:
vessel_1 = TransportProcessingResource(
    env=my_env,
    name="Vessel 01",
    ID=str(uuid.uuid4()),
    geometry=location_from_site,
    store_capacity=4,
    nr_resources=1,
    compute_v=lambda x: 10,
    initials=[
        {"id": "MP", "level": 0, "capacity": 1},
    ],
)
vessel_2 = TransportProcessingResource(
    env=my_env,
    name="Vessel 02",
    ID=str(uuid.uuid4()),
    geometry=location_from_site,
    store_capacity=4,
    nr_resources=1,
    compute_v=lambda x: 10,
    initials=[
        {"id": "MP", "level": 0, "capacity": 1},
    ],
)

In [5]:
processes = []
for i, vessel in enumerate([vessel_1,vessel_2]):
    requested_resources1 = {}
    processes.append(
        model.WhileActivity(
            env=my_env,
            name=f"while {vessel.name}",
            ID=str(uuid.uuid4()),
            registry=registry,
            sub_processes=[
                model.SequentialActivity(
                    env=my_env,
                    name=f"sequence {vessel.name}",
                    ID=str(uuid.uuid4()),
                    registry=registry,
                    sub_processes=[
                        model.MoveActivity(
                            env=my_env,
                            name=f"sailing empty {vessel.name}",
                            ID=str(uuid.uuid4()),
                            registry=registry,
                            mover=vessel,
                            destination=from_site,
                            duration=10,
                            requested_resources=requested_resources1,
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name=f"Load MP {vessel.name}",
                            ID=str(uuid.uuid4()),
                            registry=registry,
                            processor=vessel,
                            origin=from_site,
                            destination=vessel,
                            amount=1,
                            duration=10,
                            id_="MP",
                            requested_resources=requested_resources1,
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name=f"sailing filled {vessel.name}",
                            ID=str(uuid.uuid4()),
                            registry=registry,
                            mover=vessel,
                            destination=to_site,
                            duration=10,
                            requested_resources=requested_resources1,
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name=f"Transfer MP {vessel.name}",
                            ID=str(uuid.uuid4()),
                            registry=registry,
                            processor=vessel,
                            origin=vessel,
                            destination=to_site,
                            amount=1,
                            duration=10,
                            id_="MP",
                            requested_resources=requested_resources1,
                        ),
                    ],
                )
            ],
            condition_event={"type": "container","concept": from_site,"state": "empty", "id_":"MP"},
        )
    )

In [6]:
model.register_processes([
    *processes
])
my_env.run()

Winlocatie 8
Winlocatie 7
Winlocatie 6
Winlocatie 5
Winlocatie 4
Winlocatie 3
Winlocatie 2
Winlocatie 1
Winlocatie 0
Winlocatie 0


In [7]:
acts = []
for proc in processes:
    acts.extend(proc.sub_processes[0].sub_processes)
    
plot.vessel_planning([
    *acts
])